In [ ]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from multielo import MultiElo, Tracker
from mktools.get_data import load_data_pd
from mktools.validate_data import validate_bad_uids
from mktools.form_data import fill_new_session
import plotly.express as px
from alive_progress import alive_it
from bs4 import BeautifulSoup
from typing import Literal

# Load Variables from .env file
load_dotenv()

## Data Main

In [18]:
FIRST_FORM_DATA_DATE = pd.Timestamp('2024-07-17 19:15:48').tz_localize("US/Eastern").tz_convert("UTC")

In [ ]:
FIRST_FORM_DATA_DATE

In [ ]:
LAST_FORM_DATA_DATE = pd.Timestamp('1/27/2025 23:40:20').tz_localize("US/Eastern").tz_convert("UTC")

LAST_FORM_DATA_DATE

In [ ]:
# Load data_main from google sheet
df = load_data_pd(
    sheet_name="data_main",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "UID",
        "SUID",
        "NAME",
        "CHARACTER",
        "MAP",
        "PLACE",
        "PLAYERS",
        "DATE",
        "SEASON",
    ],
)

df["DATE"] = (
    pd.to_datetime(df["DATE"]).dt.tz_localize("US/Eastern").dt.tz_convert("UTC")
)

df_filtered = (
    df[(df["DATE"] < FIRST_FORM_DATA_DATE) & (df["SEASON"] >= 11)]
    .copy()
    .reset_index(drop=True)
)

df_filtered["DATE"] = df_filtered["DATE"] + pd.Timedelta(hours=8)

assert all(df_filtered["DATE"].dt.time == pd.to_datetime("12:00:00").time())

assert (
    df_filtered[["SUID", "DATE"]]
    .value_counts()
    .reset_index()
    .sort_values(by="SUID")["SUID"]
    .is_unique
)

df_filtered

In [ ]:
df_filtered.groupby(["SEASON"]).agg(count=pd.NamedAgg("UID", "nunique")).reset_index()

In [22]:
# Find UIDs that will break ELO calculation
invalid, valid = validate_bad_uids(df=df_filtered, return_valid=True)

In [ ]:
invalid

In [24]:
vdf = valid.copy()

for uid in vdf["UID"].unique():
    tdf = vdf[vdf["UID"] == uid].copy().reset_index(drop=True)

    character_cond = tdf["CHARACTER"].unique().shape[0] != tdf["PLAYERS"].max()

bad_dfs = []

for uid in vdf["UID"].unique():
    tdf = vdf[vdf["UID"] == uid].copy().reset_index(drop=True)

    name_cond = tdf["NAME"].unique().shape[0] != tdf["PLAYERS"].max()
    place_cond = tdf["PLACE"].unique().shape[0] != tdf["PLAYERS"].max()
    character_cond = tdf["CHARACTER"].unique().shape[0] != tdf["PLAYERS"].max()

    if name_cond & place_cond & character_cond:
        print("bad name, place and character")
        tdf["REASON"] = "name_place_character"
        bad_dfs.append(tdf)

    elif name_cond & place_cond & ~character_cond:
        print("bad name and place")
        tdf["REASON"] = "name_place"
        bad_dfs.append(tdf)

    elif name_cond & ~place_cond & character_cond:
        print("bad name and character")
        tdf["REASON"] = "name_character"
        bad_dfs.append(tdf)

    elif ~name_cond & place_cond & character_cond:
        print("bad place and character")
        tdf["REASON"] = "place_character"
        bad_dfs.append(tdf)

    elif name_cond:
        print("bad name")
        tdf["REASON"] = "name"
        bad_dfs.append(tdf)

    elif place_cond:
        print("bad place")
        tdf["REASON"] = "place"
        bad_dfs.append(tdf)

    elif character_cond:
        print("bad character")
        tdf["REASON"] = "character"
        bad_dfs.append(tdf)

# bad_name_data = pd.concat(bad_dfs).reset_index(drop=True)

In [25]:
vdf = valid.copy()

In [ ]:
vdf_pivot = vdf.pivot(
    index=["SEASON", "SUID", "UID", "PLAYERS", "MAP", "DATE"],
    columns=[
        "PLACE",
    ],
    values=["NAME", "CHARACTER"],
).reset_index()

# Create index of tuples for flat columns index
vdf_pivot.columns = vdf_pivot.columns.to_flat_index()
# Join tuple items together to make flat columns
vdf_pivot.columns = [f"{x[0]}{x[1]}" for x in vdf_pivot.columns]

vdf_pivot.head()

In [ ]:
# Rename pivoted columns
vdf_rename = vdf_pivot.drop(columns=["UID"]).rename(
    columns={
        "DATE": "TIMESTAMP",
        "NAME1": "PLAYERS_1ST",
        "NAME2": "PLAYERS_2ND",
        "NAME3": "PLAYERS_3RD",
        "NAME4": "PLAYERS_4TH",
        "CHARACTER1": "CHARACTERS_1ST",
        "CHARACTER2": "CHARACTERS_2ND",
        "CHARACTER3": "CHARACTERS_3RD",
        "CHARACTER4": "CHARACTERS_4TH",
    }
)

# Add new session dummy column
vdf_rename["NEW_SESSION"] = "NO"

tdf = vdf_rename.reset_index().copy()

df_holder = []

for suid in tdf["SUID"].unique():

    temp_df = tdf[tdf["SUID"] == suid].copy().reset_index(drop=True)

    first_session_timestamp = temp_df["TIMESTAMP"][0]

    temp_df["INCREMENT"] = 5

    temp_df["INCREMENT"] = temp_df["INCREMENT"].shift(fill_value=0).cumsum()

    for idx, time in enumerate(temp_df["TIMESTAMP"]):
        temp_df.loc[idx, "TIMESTAMP"] = first_session_timestamp + pd.Timedelta(
            minutes=temp_df["INCREMENT"][idx]
        )

    df_holder.append(temp_df)

tdf_concat = pd.concat(df_holder).reset_index(drop=True)

tdf_sort = tdf_concat[
    [
        "TIMESTAMP",
        "NEW_SESSION",
        "MAP",
        "PLAYERS",
        "PLAYERS_1ST",
        "PLAYERS_2ND",
        "PLAYERS_3RD",
        "PLAYERS_4TH",
        "CHARACTERS_1ST",
        "CHARACTERS_2ND",
        "CHARACTERS_3RD",
        "CHARACTERS_4TH",
    ]
].copy()

tdf_sort

## Form Data

In [28]:
form_df = load_data_pd(sheet_name="form_data", sheet_id=os.environ["SHEET_ID"])

form_df = form_df.drop(
    columns=[
        x
        for x in form_df.columns
        if x.__contains__("Unnamed") or x.__contains__("Score")
    ]
)

form_df["Timestamp"] = pd.to_datetime(form_df["Timestamp"])

form_df.columns = [
    x.replace(" ", "_").replace("[", "").replace("]", "") if x.__contains__("[") else x
    for x in form_df.columns
]

In [ ]:
def transform_form_data(
    df: pd.DataFrame, game_type_replace_string: Literal["_2_", "_3_", "_4_"]
) -> pd.DataFrame:

    idf = df.copy()

    idf_out = (
        idf.dropna(axis=1, how="all")
        .reset_index(drop=True)
        .reset_index()
        .rename(columns={"index": "ID"})
    )

    idf_out.columns = [
        (
            x.replace(game_type_replace_string, "_")
            if x.__contains__(game_type_replace_string)
            else x.upper()
        )
        for x in idf_out.columns
    ]

    idf_out["TIMESTAMP"] = (
        pd.to_datetime(idf_out["TIMESTAMP"])
        .dt.tz_localize("US/Eastern")
        .dt.tz_convert("UTC")
    )

    idf_out["ID"] = idf_out["ID"] + 1

    return idf_out


dfs = []

for game_type in [2, 3, 4]:
    tdf = form_df[form_df["PLAYERS"] == game_type].copy().reset_index(drop=True)

    dfs.append(tdf)

two_p = dfs[0]
three_p = dfs[1]
four_p = dfs[2]

two_p_out = transform_form_data(df=two_p, game_type_replace_string="_2_")
three_p_out = transform_form_data(df=three_p, game_type_replace_string="_3_")
four_p_out = transform_form_data(df=four_p, game_type_replace_string="_4_")

cat_df = (
    pd.concat([two_p_out, three_p_out, four_p_out])
    .sort_values(by="TIMESTAMP")
    .drop(columns=["ID"])
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index": "ID"})
)[
    [
        "ID",
        "TIMESTAMP",
        "NEW_SESSION",
        "MAP",
        "PLAYERS",
        "PLAYERS_1ST",
        "PLAYERS_2ND",
        "PLAYERS_3RD",
        "PLAYERS_4TH",
        "CHARACTERS_1ST",
        "CHARACTERS_2ND",
        "CHARACTERS_3RD",
        "CHARACTERS_4TH",
    ]
]

cat_df["ID"] = cat_df["ID"] + 1

cat_df

## Combine

In [ ]:
season_gb = (
    df.groupby(["DATE"])[["SEASON"]]
    .first()
    .reset_index()
    .rename(columns={"DATE": "TIMESTAMP"})
)

season_gb.tail()

In [ ]:
form_data_initial = cat_df.drop(columns=["ID"]).copy()

form_data_ready = pd.merge(
    form_data_initial, season_gb, on="TIMESTAMP", how="inner", validate="1:1"
)

# Double check that no records were dropped in the merge
assert form_data_ready.shape[0] == form_data_initial.shape[0]

form_data_ready.head()

In [ ]:
data_main_ready = tdf_sort.copy()

data_main_ready["SEASON"] = 11

data_main_ready.tail()

In [ ]:
data_concat = (
    pd.concat([data_main_ready, form_data_ready])
    .sort_values(by="TIMESTAMP")
    .reset_index(drop=True)
)

data_concat

In [ ]:
data_concat["SEASON"].value_counts()

In [ ]:
data_concat["TIMESTAMP"].is_unique

In [ ]:
data_concat["TIMESTAMP"].is_monotonic_increasing

In [ ]:
data_concat["TIMESTAMP"].is_monotonic_decreasing

In [ ]:
data_concat_filled = fill_new_session(
    df=data_concat, timestamp_column_name="TIMESTAMP", drop_window_start_column=True
)

# Convert back to EST for Postgres
data_concat_filled["TIMESTAMP"] = data_concat_filled["TIMESTAMP"].dt.tz_convert(
    "US/Eastern"
)

data_concat_filled

In [ ]:
data_concat_filled[["SEASON", "NEW_SESSION"]].value_counts()

In [ ]:
data_concat_filled["TIMESTAMP"]

In [50]:
new_session_df = data_concat_filled.copy()

new_session_df["temp_session"] = np.where(new_session_df["NEW_SESSION"] == "YES", 1, 0)

new_session_df["SUID"] = new_session_df["temp_session"].cumsum()

In [ ]:
new_session_df.columns

In [55]:
out_df = new_session_df[
    [
        "TIMESTAMP",
        "NEW_SESSION",
        "SUID",
        "MAP",
        "PLAYERS",
        "PLAYERS_1ST",
        "PLAYERS_2ND",
        "PLAYERS_3RD",
        "PLAYERS_4TH",
        "CHARACTERS_1ST",
        "CHARACTERS_2ND",
        "CHARACTERS_3RD",
        "CHARACTERS_4TH",
        "SEASON",
    ]
].copy()

In [56]:
out_df.to_csv(
    rf"C:\Users\Cooper\sandbox\mkstream\form_data_migration\form_data_valid.csv",
    index=False,
)

## images

In [60]:
import os

map_icons = [f"/maps/{x}" for x in os.listdir(rf"\\wsl$\Ubuntu\root\learn\mk\public\maps")]

In [ ]:
pd.Series(out_df["MAP"].unique()).to_list()

In [ ]:
map_names_sorted = (
    pd.Series(
        [
            "Wario Stadium",
            "D.K.'s Jungle",
            "Sherbet Land",
            "Koopa Troopa Beach",
            "Yoshi Valley",
            "Banshee Boardwalk",
            "Royal Raceway",
            "Kalimari Desert",
            "Bowser's Castle",
            "Toad's Turnpike",
            "Frappe Snowland",
            "Choco Mountain",
            "Mario Raceway",
            "Moo Moo Farm",
            "Luigi Raceway",
            "Rainbow Road",
        ]
    )
    .sort_values()
    .reset_index(drop=True)
)

In [ ]:
map_paths_sorted = pd.Series(map_icons).sort_values().reset_index(drop=True)

In [ ]:
maps_df = pd.DataFrame(
    {
        "MAP": map_names_sorted,
        "IMAGE_URL": map_paths_sorted,
    }
)

maps_df

In [ ]:
maps_df.to_csv(
    rf"C:\Users\Cooper\sandbox\mkstream\form_data_migration\maps_valid.csv",
    index=False,
)